In [1]:
#!/usr/bin python3.6
import os
import sys
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
sys.path.append("/home/mabon/Tiny_power/code/TinyPower")

import checking_tool
import process_data
from utils import model_zoo

2023-07-08 18:34:03.469837: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
model = load_model('/home/mabon/Tiny_power/models/original/HW/stm32/S1/model/best_model.h5')
model.summary()
model_pruned= model_zoo.cnn_best(input_shape=(1000, 1), emb_size=9, classification=True)
model_pruned.summary()

2023-07-08 18:34:04.580084: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-08 18:34:04.580508: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-08 18:34:04.658482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-08 18:34:04.659142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2023-07-08 18:34:04.659157: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-07-08 18:34:04.660416: I tensorflow/stream_executor/platform/default/dso_load

Model: "cnn_best"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 1)]         0         
_________________________________________________________________
block1_conv1 (Conv1D)        (None, 500, 64)           768       
_________________________________________________________________
block1_pool (AveragePooling1 (None, 250, 64)           0         
_________________________________________________________________
block2_conv1 (Conv1D)        (None, 250, 128)          90240     
_________________________________________________________________
block2_pool (AveragePooling1 (None, 125, 128)          0         
_________________________________________________________________
block3_conv1 (Conv1D)        (None, 125, 256)          360704    
_________________________________________________________________
block3_pool (AveragePooling1 (None, 62, 256)           0  

In [3]:
mini_weights0 = []
mini_weights1 = []

for j,layer in enumerate(model_pruned.layers):
    weights = layer.get_weights()
    if ("conv" or "dense" in layer.name) and j != (len(model_pruned.layers) - 1) :  # Fix the attribute name from "layer.layer" to "layer.name"
        #print(f"Layer: {layer.name}")
        for i, weight in enumerate(weights):
            print(f"Weight {i}: {weight.shape}")
            if i == 0:
                mini_weights0.append(weight)
            else:
                mini_weights1.append(weight)

Weight 0: (11, 1, 64)
Weight 1: (64,)
Weight 0: (11, 64, 128)
Weight 1: (128,)
Weight 0: (11, 128, 256)
Weight 1: (256,)
Weight 0: (11, 256, 512)
Weight 1: (512,)
Weight 0: (11, 512, 512)
Weight 1: (512,)
Weight 0: (7680, 4096)
Weight 1: (4096,)
Weight 0: (4096, 4096)
Weight 1: (4096,)



In most cases, a layer in a neural network consists of two types of weights: weight 0 and weight 1. The interpretation of these weights depends on the type of layer.

Dense Layer (Fully Connected Layer):

Weight 0: This corresponds to the weight matrix that connects the inputs from the previous layer to the current layer.
Weight 1: This represents the bias vector added to the outputs of the weighted inputs.
Convolutional Layer:

Weight 0: This represents the set of filters or kernels used for convolution with the input.
Weight 1: This corresponds to the bias vector added to the outputs of the convolution operation.


In [4]:
weights0 = []
weights1 = []

for j,layer in enumerate(model.layers):
    weights = layer.get_weights()
    if ("conv" or "dense" in layer.name) and j != (len(model.layers) - 1) :  # Fix the attribute name from "layer.layer" to "layer.name"
        #print(f"Layer: {layer.name}")
        for i, weight in enumerate(weights):
            print(f"Weight {i}: {weight.shape}")
            if i == 0:
                weights0.append(weight)
            else:
                weights1.append(weight)


Weight 0: (11, 1, 64)
Weight 1: (64,)
Weight 0: (11, 64, 128)
Weight 1: (128,)
Weight 0: (11, 128, 256)
Weight 1: (256,)
Weight 0: (11, 256, 512)
Weight 1: (512,)
Weight 0: (11, 512, 512)
Weight 1: (512,)
Weight 0: (7680, 4096)
Weight 1: (4096,)
Weight 0: (4096, 4096)
Weight 1: (4096,)


In [5]:
def replace_lowest_percent(arr, x):
    num_elements = len(arr)
    num_elements_to_replace = int(num_elements * (x / 100))  # Calculate the number of elements to replace
    sorted_indices = np.argsort(arr)  # Sort the indices based on the array values

    # Replace the lowest x% of elements with 0
    arr[sorted_indices[:num_elements_to_replace]] = 0

    return arr

In [6]:
# Example usage
x = 50  # Percentage of elements to replace with 0
for i in range(len(weights1)):
    weights1[i]=replace_lowest_percent(weights1[i], x)


In [7]:
len(weights1)

7

In [17]:
def copy_modified_weights(weights1,source_model, target_model):
    i=0
    for source_layer, target_layer in zip(source_model.layers, target_model.layers):
        source_weights = source_layer.get_weights()
        target_weights = target_layer.get_weights()
        if ("conv" in target_layer.name ) and target_layer != target_model.layers[-1]:
            source_weights[1]=weights1[i]
            i=i+1
            target_layer.set_weights(source_weights)
            #print("flag")
            print(np.shape(source_weights[1]))
        elif "Dense" == (source_layer.__class__.__name__ )and target_layer != target_model.layers[-1]:
            source_weights[1] = weights1[i]
            i += 1
            target_layer.set_weights(source_weights)
            print(np.shape(source_weights[1]))
        else:
            target_layer.set_weights(source_weights)
    return target_model


In [18]:
model_pruned=copy_modified_weights(weights1,model, model_pruned)

(64,)
(128,)
(256,)
(512,)
(512,)
(4096,)
(4096,)


In [10]:
mini_weights2 = []
mini_weights3 = []

for j,layer in enumerate(model_pruned.layers):
    weights = layer.get_weights()
    if ("conv" or "dense" in layer.name) and j != (len(model_pruned.layers) - 1) :  # Fix the attribute name from "layer.layer" to "layer.name"
        #print(f"Layer: {layer.name}")
        for i, weight in enumerate(weights):
            print(f"Weight {i}: {weight.shape}")
            if i == 0:
                mini_weights2.append(weight)
            else:
                mini_weights3.append(weight)

Weight 0: (11, 1, 64)
Weight 1: (64,)
Weight 0: (11, 64, 128)
Weight 1: (128,)
Weight 0: (11, 128, 256)
Weight 1: (256,)
Weight 0: (11, 256, 512)
Weight 1: (512,)
Weight 0: (11, 512, 512)
Weight 1: (512,)
Weight 0: (7680, 4096)
Weight 1: (4096,)
Weight 0: (4096, 4096)
Weight 1: (4096,)
